# Tanpopo1 表面付着物 EfficientNet

In [1]:
# Google Colab マウント
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive
import os
os.chdir('/content/drive/MyDrive/Tanpopo')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [2]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torch.utils.data as data

import matplotlib.pyplot as plt
import glob
import time
import copy
from PIL import Image

plt.ion()

In [3]:
#画像サイズがが704x480 #88x60
img_size = 224

#class_names = ['1Sputter', '2Fiber', '3Block', '4Bar', '5AGFragment']
class_num = 5

# 標準化
mean = (0.5, 0.5, 0.5)
std = (0.5, 0.5, 0.5)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [4]:
batch_size =  "16" #@param[8, 16, 32, 64, 128, 256]
batch_size = int(batch_size)

epochs = "22" #@param[5, 8, 10, 15, 20, 22, 25, 27, 29, 30, 31, 32, 33, 35, 45, 60, 120]
epochs = int(epochs)

### 関数、クラスの作成

In [5]:
import random
from sklearn.model_selection import train_test_split
def make_filepath_list(folderpath, phase='train'):
    """
    ファイルのパスを格納したリストを返す
    """
    file_list = []
    test_file_list = []
    train_file_list = []
    valid_file_list = []
    class_names = []

    # .DS_Storeが最初に読み込まれる
    for index, top_dir in enumerate(sorted(os.listdir(folderpath))):
        file_dir = os.path.join(folderpath, top_dir)
        file_list = glob.glob(file_dir + '/*bmp')

        if top_dir != '.DS_Store':
            class_names.append(top_dir)

            if phase == 'test': # テストデータの場合
                test_file_list += [os.path.join(folderpath, top_dir, file).replace('\\', '/') for file in file_list]
                                                            
            else:
                # 各クラス(フォルダ)ごとに8割を訓練データ、2割を検証データとする
                file_list = [os.path.join(folderpath, top_dir, file).replace('\\', '/') for file in file_list]
                train_file, valid_file = train_test_split(file_list, train_size=0.8, test_size=0.2, random_state=0)
                train_file_list += train_file
                valid_file_list += valid_file

    if phase == 'test':
        return test_file_list, class_names

    return train_file_list, valid_file_list, class_names

In [6]:
from torchvision.io import read_image

class SurfaceObjectDataset(data.Dataset):
    """
    表面付着物のDatasetクラス
    PyTorchのDatasetクラスを継承
    """
    def __init__(self, file_list, classes, transform=None, phase='train'):
        #super().__init__()
        self.file_list = file_list
        self.transform = transform
        self.classes = classes
        self.phase = phase

        self.img = None
        self.label = None

    def __len__(self):
        """
        画像の枚数を返す
        """
        return len(self.file_list)

    def __getitem__(self, index):
        """
        前処理した画像データのTensor形式のデータとラベルを取得
        """
        # 指定したindexの画像を読み込む
        img_path = self.file_list[index]
        img = Image.open(img_path)

        # 画像ラベルをファイル名から抜き出す
        label = self.file_list[index].split('/')[6][:11]

        # ラベル名を数値に変換
        label = self.classes.index(label)

        # 画像の前処理を実施
        if self.transform is not None:
            img_transformed = self.transform(img, self.phase)
        
        self.img = img_transformed
        self.label = label

        return img_transformed, label

In [7]:
class ImageTransform(object):
    """
    画像の前処理
    """
    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                # データオグメンテーション, 前処理
                transforms.Resize(256), # リサイズ
                transforms.CenterCrop(resize), # 切り取り
                transforms.RandomRotation(45), # ランタムに回転
                transforms.ColorJitter(), # ランダムに明るさ、コントラスト、彩度、色相を変化
                transforms.RandomHorizontalFlip(), #ランダムに左右(水平)反転
                transforms.RandomVerticalFlip(), # ランダムに上下(垂直)反転
                transforms.ToTensor(),
                transforms.Normalize(mean, std), # zcaと交換？
                # ZCA whitening追加する
            ]),
            'valid': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std),
            ]),
            'test': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std),
            ]),
            'gray': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std),
                transforms.Grayscale()
            ])
        }

    def __call__(self, img, phase='train'):
        return self.data_transform[phase](img)

### データ読み込み・前処理

In [8]:
import torch.utils.data as data
# 訓練、検証データへのファイルパスを格納したリストを取得
train_file_list, valid_file_list, class_names = make_filepath_list('/content/drive/MyDrive/Tanpopo/TrainingData11', 'train')

print('train_file_list: ', train_file_list)
print('class_names: ', class_names)
class_num = len(class_names) # 5

train_each_class_num = dict()
for class_name in class_names:
    train_each_class_num[class_name] = 0

valid_each_class_num = dict()
for class_name in class_names:
    valid_each_class_num[class_name] = 0

for file in train_file_list: # train class num
    label = file.split('/')[6][:11]
    train_each_class_num[label] += 1

for file in valid_file_list: # validation class num
    label = file.split('/')[6][:11]
    valid_each_class_num[label] += 1

print('train_each_class_num: ', train_each_class_num)
print('valid_each_class_num: ', valid_each_class_num)

# Datasetの作成
train_dataset = SurfaceObjectDataset(
    file_list = train_file_list, classes = class_names,
    transform = ImageTransform(img_size, mean, std),
    phase = 'train'
)

valid_dataset = SurfaceObjectDataset(
    file_list = valid_file_list, classes = class_names,
    transform = ImageTransform(img_size, mean, std),
    phase = 'valid'
)

# Dataloaderの作成
train_dataloader = data.DataLoader(
    train_dataset, batch_size = batch_size, shuffle=True
)

valid_dataloader = data.DataLoader(
    valid_dataset, batch_size = int(batch_size/2), shuffle=False
)

dataloaders_dict = {
    'train': train_dataloader,
    'valid': valid_dataloader
}

# デフォルトでは `log_dir` としていますが、今回は "runs" とさらに具体的な実験名まで記載します
#writer = SummaryWriter('/content/drive/MyDrive/Tanpopo/runs/surfaceObject_experiment_1')

# # イテレータに変換
# batch_iterator = iter(dataloaders_dict['train'])

# inputs, labels = next(batch_iterator)f

train_file_list:  ['/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SG3A0024S000.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SM3A0157S008.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SM3A0185S007.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SM3A0083S008.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SP3A0279S011.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SF3A0035S003.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SP3A0257S000.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SG3A0199S003.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SG3A0060S000.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SM3A0094S008.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SM3A0025S000.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SG3A0088S000.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SP3A0206S000.bmp', '/cont

### モデルの作成

In [9]:
# モデルをロード
# EfficientNet学習済みの重みを使用
model_efficientnet = models.efficientnet_b0(pretrained=True)
print(model_efficientnet)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
#print(model_vgg16)

In [11]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
#from torchsummary import summary
from torchinfo import summary
model_efficientnet.to(device)
print(summary(model=model_efficientnet, input_size=(batch_size, 3, 224, 224),
        col_names=['input_size', 'output_size', 'num_params'], 
        device=device,
        #verbose=2
))
print('EfficientNet')

Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #
EfficientNet                                            [16, 3, 224, 224]         [16, 1000]                --
├─Sequential: 1-1                                       [16, 3, 224, 224]         [16, 1280, 7, 7]          --
│    └─Conv2dNormActivation: 2-1                        [16, 3, 224, 224]         [16, 32, 112, 112]        --
│    │    └─Conv2d: 3-1                                 [16, 3, 224, 224]         [16, 32, 112, 112]        864
│    │    └─BatchNorm2d: 3-2                            [16, 32, 112, 112]        [16, 32, 112, 112]        64
│    │    └─SiLU: 3-3                                   [16, 32, 112, 112]        [16, 32, 112, 112]        --
│    └─Sequential: 2-2                                  [16, 32, 112, 112]        [16, 16, 112, 112]        --
│    │    └─MBConv: 3-4                                 [16, 32, 112, 112]        [16, 16, 112, 112]      

In [13]:
import time
import copy

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = 10.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))

        # 各エポックには訓練フェーズと検証フェーズがあります
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # モデルを訓練モードに設定します
            else:
                model.eval()   # モードを評価するモデルを設定します

            running_loss = 0.0
            running_corrects = 0

            # データをイレテートします
            for inputs, labels in dataloaders_dict[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # パラメータの勾配をゼロにします
                optimizer.zero_grad()

                # 順伝播
                # 訓練の時だけ、履歴を保持します
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # 訓練の時だけ逆伝播＋オプティマイズを行います
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # 損失を計算します
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders_dict[phase].dataset)

            print(' {} Loss: {:.4f} Acc: {:.4f} '.format(
                phase, epoch_loss, epoch_acc), end='\t')

            # モデルをディープ・コピー
            if phase == 'valid' and epoch_acc >= best_acc:
                best_acc = epoch_acc
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    print('Best val Acc: {:4f}'.format(best_loss))

    # ベストモデルの重みをロード
    model.load_state_dict(best_model_wts)
    return model

In [14]:
# EfficientNet の最後の層
model_efficientnet.classifier[1] = nn.Linear(in_features=1280, out_features=5)

# 転移学習で学習させるパラメータを、変数params_to_updateに格納
params_to_update = []

# まず全パラメータを勾配計算Falseにする
for name, param in model_efficientnet.named_parameters():
    param.requires_grad = False

# 全結合層のみパラメータ更新
for name, param in model_efficientnet.classifier.named_parameters():
    param.requires_grad = True
    params_to_update.append(param)

optimizer = optim.AdamW(params_to_update, lr=0.1, weight_decay=0.8)

criterion = nn.CrossEntropyLoss()

# 7エポックごとに学習率を1/10ずつ減衰させます
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

### 学習

In [15]:
PATH = '/content/drive/MyDrive/Tanpopo/model_efficientnet_weights.pth'
#model_efficientnet.load_state_dict(torch.load(PATH)) # 学習前: 前回の重みを使う

In [16]:
epochs = 36
# 学習
print('EfficientNet Training :')
print('-'*10)
model_efficientnet = model_efficientnet.to(device)
model_efficientnet = train_model(model_efficientnet, criterion, optimizer, scheduler, num_epochs=epochs)

EfficientNet Training :
----------
Epoch 0/35
 train Loss: 8.2146 Acc: 0.2923 	 valid Loss: 2.5644 Acc: 0.5800 	
Epoch 1/35
 train Loss: 3.9970 Acc: 0.4923 	 valid Loss: 1.7432 Acc: 0.6200 	
Epoch 2/35
 train Loss: 2.9109 Acc: 0.4103 	 valid Loss: 0.9058 Acc: 0.7400 	
Epoch 3/35
 train Loss: 2.5459 Acc: 0.4718 	 valid Loss: 1.4149 Acc: 0.5800 	
Epoch 4/35
 train Loss: 2.2379 Acc: 0.4000 	 valid Loss: 2.4866 Acc: 0.4600 	
Epoch 5/35
 train Loss: 2.8232 Acc: 0.4821 	 valid Loss: 1.5726 Acc: 0.5600 	
Epoch 6/35
 train Loss: 2.0261 Acc: 0.5179 	 valid Loss: 1.5838 Acc: 0.5200 	
Epoch 7/35
 train Loss: 1.9311 Acc: 0.5436 	 valid Loss: 1.6382 Acc: 0.5000 	
Epoch 8/35
 train Loss: 1.5288 Acc: 0.5692 	 valid Loss: 1.2043 Acc: 0.6200 	
Epoch 9/35
 train Loss: 1.3592 Acc: 0.5590 	 valid Loss: 1.1100 Acc: 0.6600 	
Epoch 10/35
 train Loss: 1.0171 Acc: 0.6256 	 valid Loss: 1.1305 Acc: 0.5800 	
Epoch 11/35
 train Loss: 0.9003 Acc: 0.6513 	 valid Loss: 1.0061 Acc: 0.6400 	
Epoch 12/35
 train Loss: 1.

In [ ]:
PATH = '/content/drive/MyDrive/Tanpopo/model_efficientnet_weights.pth'
#torch.save(model_efficientnet.state_dict(), PATH) # 重みを保存

### テスト

In [17]:
import torch.utils.data as data
PATH = '/content/drive/MyDrive/Tanpopo/model_efficientnet_weights.pth'
#model_efficientnet.load_state_dict(torch.load(PATH))

# テストデータ
test_file_list, class_names_test = make_filepath_list('/content/drive/MyDrive/Tanpopo/TestData11', 'test')

print('test_file_list : ', test_file_list)
print('class_names_test : ', class_names_test)

# Datasetの作成
test_dataset = SurfaceObjectDataset(
    file_list = test_file_list, classes = class_names_test,
    transform = ImageTransform(img_size, mean, std),
    phase = 'test'
)

# Dataloaderの作成
test_dataloader = data.DataLoader(
    test_dataset, batch_size = int(batch_size/2), shuffle=False
)

dataloaders_dict['test'] = test_dataloader

test_file_list :  ['/content/drive/MyDrive/Tanpopo/TestData11/1Sputter/SF3A0065S008.bmp', '/content/drive/MyDrive/Tanpopo/TestData11/1Sputter/SF3A0100S010.bmp', '/content/drive/MyDrive/Tanpopo/TestData11/1Sputter/SG3A0020S000.bmp', '/content/drive/MyDrive/Tanpopo/TestData11/1Sputter/SG3A0041S000.bmp', '/content/drive/MyDrive/Tanpopo/TestData11/1Sputter/SG3A0053S000.bmp', '/content/drive/MyDrive/Tanpopo/TestData11/1Sputter/SG3A0058S000.bmp', '/content/drive/MyDrive/Tanpopo/TestData11/1Sputter/SG3A0067S000.bmp', '/content/drive/MyDrive/Tanpopo/TestData11/1Sputter/SG3A0089S000.bmp', '/content/drive/MyDrive/Tanpopo/TestData11/1Sputter/SG3A0096S000.bmp', '/content/drive/MyDrive/Tanpopo/TestData11/1Sputter/SG3A0098S000.bmp', '/content/drive/MyDrive/Tanpopo/TestData11/1Sputter/SG3A0102S000.bmp', '/content/drive/MyDrive/Tanpopo/TestData11/1Sputter/SG3A0111S000.bmp', '/content/drive/MyDrive/Tanpopo/TestData11/1Sputter/SL3A0000S000.bmp', '/content/drive/MyDrive/Tanpopo/TestData11/1Sputter/SM3A00

In [18]:
from sklearn.metrics import confusion_matrix
import numpy as np

#正解率
def class_accuracy(label, conf_mat):
    return (conf_mat[label][label] + (np.sum(conf_mat) - (np.sum(conf_mat[:, label])+np.sum(conf_mat[label])-conf_mat[label][label]))) / np.sum(conf_mat)
    
#精度(適合率)
def class_precision(label, conf_mat):
    return conf_mat[label][label] / np.sum(conf_mat[label])

#再現率
def class_recall(label, conf_mat):
    return conf_mat[label][label] / np.sum(conf_mat[:, label])

labels_sum = None
predicted_sum = None

with torch.no_grad():
    for data in dataloaders_dict['test']:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)

        model_efficientnet.eval()
        outputs = model_efficientnet(images)

        #outputs = nn.Softmax(dim=1)(outputs)

        _, predicted = torch.max(outputs, 1)

        if labels_sum is None:
            labels_sum = labels
            predicted_sum = predicted
        else:
            labels_sum = torch.cat([labels_sum, labels], dim=0)
            predicted_sum = torch.cat([predicted_sum, predicted], dim=0)

In [19]:
#混同行列
labels_sum = labels_sum.cpu()
predicted_sum = predicted_sum.cpu()
conf_mat = None
print('EfficientNet network')
print(class_names_test)

Accuracy = []
Precision = []
Recall = []

conf_mat = confusion_matrix(labels_sum, predicted_sum)
print(conf_mat)
print()
for i in range(class_num):
    Accuracy = np.append(Accuracy, class_accuracy(i, conf_mat)*100)
    Precision = np.append(Precision, class_precision(i, conf_mat)*100)
    Recall = np.append(Recall, class_recall(i, conf_mat)*100)

np.set_printoptions(precision=1)

print('Accuracy : ', Accuracy)
print('Recall : ', Recall)
print('Precision : ', Precision)

EfficientNet network
['1Sputter', '2Fiber', '3Block', '4Bar', '5AGFragment']
[[18  2  1  0  0]
 [ 1 17  1  0  2]
 [ 1  1 16  3  0]
 [ 1  3  0 17  0]
 [ 2  4  2  0 13]]

Accuracy :  [92.4 86.7 91.4 93.3 90.5]
Recall :  [78.3 63.  80.  85.  86.7]
Precision :  [85.7 81.  76.2 81.  61.9]
